In [139]:
import json
import time
import requests
import hashlib
import csv
import pandas as pd
from datetime import datetime


In [141]:
class DomClickApi:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({"X-Service": "true",
                                    "Connection": "Keep-Alive",
                                     "User-Agent": "Android; 12; Google; google_pixel_5; 8.72.0; 8720006; ; NONAUTH"
                                     })

        # init (get cookies)
        self.get("https://api.domclick.ru/core/no-auth-zone/api/v1/ensure_session")
        self.get("https://ipoteka.domclick.ru/mobile/v1/feature_toggles")

    def get(self, url, **kwargs):
        self.__update_headers(url, **kwargs)
        result = self.session.get(url, **kwargs)
        #print(self.session.cookies.get_dict())
        return result

    def __update_headers(self, url, **kwargs):
        url = self.__get_prepared_url(url, **kwargs)
        sault = "ad65f331b02b90d868cbdd660d82aba0"
        timestamp = str(int(datetime.now().timestamp()))
        encoded = (sault + url + timestamp).encode("UTF-8")
        h = hashlib.md5(encoded).hexdigest()
        self.session.headers.update({"Timestamp": timestamp,
                                     "Hash": "v1:" + h,
                                     })

    def __get_prepared_url(self, url, **kwargs):
        p = requests.models.PreparedRequest()
        p.prepare(method="GET", url=url, **kwargs)
        return p.url



In [142]:

def pprint_json(json_str):
    try:
        json_object = json.loads(json_str)
        json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False).encode('utf8')
        print(json_formatted_str.decode())
    except:
        pass
        print(json_str)


In [4]:
def get_keys(d):
    if isinstance(d, dict):
        for k, v in d.items():
            yield k
            yield from list(get_keys(v))
    elif isinstance(d, list):
        for o in d:
            yield from list(get_keys(o))

In [ ]:
remont = ['design', 'without_repair', 'standard', 'well_done'] #without_repair design standard well_done
balcons = [1, 2]
rooms = [1, 2, 3]
type_dome = ['loyout', 'flat'] #flat loyout
perec = ['panel', 'brick', 'panel', 'block'] #monolith  brick panel block

offers_url = 'https://offers-service.domclick.ru/research/v5/offers/'
count_url = 'https://offers-service.domclick.ru/research/v5/offers/count/'

dca = DomClickApi()
for type in perec:
    for flat in rooms:
        for sd in type_dome:
            for rem in remont:
                for balcon in balcons:
                    res = dca.get(count_url, params={
                        "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
                        "deal_type": "sale",
                        "category": "living",
                        "offer_type": ["flat", "layout"],
                        "rooms": flat,
                        'renovation': rem,
                        "wall_type": type,
                        "balconies": balcon
                    })
                    print("RES:", res)
                    print(res.text)
                    pprint_json(res.text)

                    count_obj = json.loads(res.text)
                    total = count_obj["pagination"]["total"]


In [ ]:
#def get_seo_info(J_data):
#    for j in J_data['items']:
#        
   

In [150]:
offset = 0
df_data = []
try:
    for type in perec:
        for flat in rooms:
            for sd in type_dome:
                for rem in remont:
                    for balcon in balcons:
                        #print(balcon)
                        for offset in range(0, 1999, 10):
                            res = dca.get(offers_url, params={
                                    "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
                                    "deal_type": "sale",
                                    "category": "living",
                                    "offer_type": ["flat", "layout"],
                                    "offset": offset,
                                    "limit": 10,
                                    "rooms": flat,
                                    "wall_type": type,
                                    "renovation": rem,
                                    'balconies': balcon
                            })
                                #print("RES:", res)
                                #pprint_json(res.text)
                            offers_obj = json.loads(res.text)
                            #print(list(get_keys(offers_obj)))
                            total = offers_obj["pagination"]["total"]
                            result_data = offers_obj['result']#['items']
                            items = result_data['items']
                            for item in items:
                                    address = item['address']
                                    price = item['price_info']
                                    house = item['house']
                                    object_info = item['object_info']
                                    print(result_data[])
                                    #renovation = item['renovation']
                                    row = {
                                            #'address_id': address['id'],
                                            'address_name': address['name'],
                                            'price': price['price'],
                                            'square_price': price['square_price'],
                                            'floors': house['floors'],
                                            'total_floors': house['floors'],
                                            #'chat_avail': item['chat_available'],
                                            'rooms': object_info['rooms'],
                                            'area': object_info['area'],
                                            'local': address['locality']['name'],
                                            #'type_house': type,
                                            'type_home': sd,
                                            'remont': rem,
                                            'balcon': balcon,
                                            #'year': result_data['items'][0]

                                    }
                                        # Находим seo_uri для данной станции метро
                                        #for seo_subway in seo_info['subways']:
                                        #    if seo_subway['guid'] == subway['guid']:
                                        #        row['seo_uri'] = seo_subway['seo_uri']
                                        #        break

                                        # Добавляем словарь в список данных
                                    df_data.append(row)
                            df = pd.DataFrame(df_data)
except ConnectionError:
     print("error")
df.to_csv('12.csv')
                            
#df = df.drop_duplicates(subset=['address_name'])
                            


KeyError: 0

In [ ]:
df = pd.read_csv('12.csv')
df

In [146]:
df

,address_name,price,square_price,floors,total_floors,rooms,area,local,type_home,remont,balcon
0,"улица Липчанского, 6",10400000,266666,17,17,1,39.0,Москва,loyout,design,1
1,"Полоцкая улица, 25к1",16400000,431578,16,16,1,38.0,Москва,loyout,design,1
2,"Алма-Атинская улица, 8к1",13950000,367105,14,14,1,38.0,Москва,loyout,design,1
3,"улица Маршала Голованова, 13",11900000,316489,17,17,1,37.6,Москва,loyout,design,1
4,"улица Введенского, 20к1",13300000,380000,9,9,1,35.0,Москва,loyout,design,1
...,...,...,...,...,...,...,...,...,...,...,...
9909,"Перовская улица, 6к2",17330000,266205,12,12,3,65.1,Москва,flat,without_repair,2
9910,"1-й микрорайон, 8",12450000,179136,14,14,3,69.5,Московский,flat,without_repair,2
9911,"улица Зои и Александра Космодемьянских, 7к4",18000000,278207,14,14,3,64.7,Москва,flat,without_repair,2
9912,"Стартовая улица, 27к1",15000000,232919,14,14,3,64.4,Москва,flat,without_repair,2


In [5]:
main = pd.read_csv('data/1_data.csv')
#main = pd.concat([df, main], ignore_index=True)
#main.to_csv('1_data.csv')

In [737]:
#main = main.drop_duplicates(subset=['address_name'])

In [14]:
main = main.drop(['Unnamed: 0.58'], axis = 1, )

In [16]:

main.to_csv('1_data.csv')


In [17]:
main

,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon
0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2
1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2
2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2
3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2
4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...
35470,"Детский проезд, 1к1",6400000,118518,19,2,54.00,Казань,monolith,flat,standard,19,1
35471,"Нижняя улица, 8",7800000,95823,17,3,81.40,Казань,monolith,flat,standard,17,1
35472,"улица Рауиса Гареева, 109",5650000,200996,23,0,28.11,Казань,monolith,flat,standard,23,1
35473,"улица Нурихана Фаттаха, 17/4",6000000,92307,18,2,65.00,Казань,monolith,flat,standard,18,1
